<a href="https://colab.research.google.com/github/EmelyanovAndreyNSK/PythonProjects/blob/master/ensemble_detections_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ensemble-boxes

In [ ]:
from ensemble_boxes import *

In [ ]:
import pickle

In [ ]:
# Load the predictions from the pickle file
with open('/content/predictions_faster_rcnn_g.pkl', 'rb') as f:
    faster_rcnn_predictions = pickle.load(f)

In [ ]:
# Load the predictions from the pickle file
with open('/content/predictions_retinanet_g.pkl', 'rb') as f:
    retinanet_predictions = pickle.load(f)

In [ ]:
# Load the predictions from the pickle file
with open('/content/predictions_ssd_g.pkl', 'rb') as f:
    ssd_predictions = pickle.load(f)

In [ ]:
#faster_rcnn_predictions[139]

In [ ]:
ensemble_predictions = {}

iou_thr = 0.5
skip_box_thr = 0.0001
sigma = 0.1

weights = [1, 1, 1]

for img_id in faster_rcnn_predictions:
    boxes_list =  [faster_rcnn_predictions[img_id]['bboxes'], ssd_predictions[img_id]['bboxes'], retinanet_predictions[img_id]['bboxes']]
    labels_list = [faster_rcnn_predictions[img_id]['labels'], ssd_predictions[img_id]['labels'], retinanet_predictions[img_id]['labels']]
    scores_list = [faster_rcnn_predictions[img_id]['scores'], ssd_predictions[img_id]['scores'], retinanet_predictions[img_id]['scores']]

    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    #boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)

    # Convert the NumPy array to a list
    list_labels = labels.tolist()
    # Convert each element in the list to an integer
    integer_list_labels = [int(i) for i in list_labels]

    # Конвертируем координаты ограничивающих рамок обратно в пиксели
    image_height, image_width = 426, 640

    boxes[:, 0] *= image_width
    boxes[:, 1] *= image_height
    boxes[:, 2] *= image_width
    boxes[:, 3] *= image_height

    ensemble_predictions[img_id] = [boxes, integer_list_labels, scores]

    #print(img_id, boxes, integer_list_labels, scores)

/usr/local/lib/python3.10/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:63: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/usr/local/lib/python3.10/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:51: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/usr/local/lib/python3.10/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:36: UserWarning: X2 < X1 value in box. Swap them.
  warnings.warn('X2 < X1 value in box. Swap them.')
/usr/local/lib/python3.10/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:39: UserWarning: Y2 < Y1 value in box. Swap them.
  warnings.warn('Y2 < Y1 value in box. Swap them.')


In [ ]:
print(ensemble_predictions[139])

[array([[291.41454697, 216.71519423, 352.84812927, 317.57244909],
       [371.18232727, 216.47143507, 425.65906525, 316.72470045],
       [  6.43368483, 168.40824902, 154.72437859, 264.79496098],
       [230.89775085, 174.77999908, 266.76708221, 213.83604312],
       [425.25241852, 159.84545964, 464.84600067, 299.49057269],
       [448.84082794, 121.13229078, 460.82958221, 142.48007011],
       [242.06199646, 198.28292674, 253.63092422, 212.61709499],
       [166.88640594, 233.10431564, 186.05657578, 266.98244548],
       [465.7964325 , 351.73697054, 637.27588654, 420.53253293],
       [412.46135712, 219.77226412, 440.9733963 , 306.8662169 ],
       [334.25281525, 178.05462402, 368.7292099 , 225.98827493],
       [351.1013031 , 206.13558984, 361.52446747, 231.41816247],
       [491.86698914, 166.31919497, 513.36341858, 277.37120676],
       [445.95443726, 167.36724508, 511.52610779, 289.46465707],
       [360.82618713, 214.30786943, 373.67549896, 231.29407382],
       [550.44883728, 29

In [ ]:
!pip install pycocotools requests

In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import os
import skimage.io as io

import requests

import zipfile

In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import json
import cv2

In [ ]:
# URL для загрузки изображений
image_url = "http://images.cocodataset.org/zips/val2017.zip"

# Загрузка изображений
response = requests.get(image_url)
with open("val2017.zip", "wb") as f:
    f.write(response.content)

In [ ]:
# URL для загрузки аннотаций
annotation_url = "http://images.cocodataset.org/annotations/annotations_trainval2017.zip"

# Загрузка аннотаций
response = requests.get(annotation_url)
with open("annotations_trainval2017.zip", "wb") as f:
    f.write(response.content)

In [ ]:
# Распаковка аннотаций
with zipfile.ZipFile("annotations_trainval2017.zip", "r") as zip_ref:
    zip_ref.extractall("annotations")

# Распаковка изображений
with zipfile.ZipFile("val2017.zip", "r") as zip_ref:
    zip_ref.extractall("images")

In [ ]:
# Загрузка аннотаций для тестовой выборки COCO
coco = COCO('/content/annotations/annotations/instances_val2017.json')

# Получение списка ID изображений в тестовой выборке
#image_ids = coco.getImgIds()
# Получаем список всех ключей словаря
image_ids = list(faster_rcnn_predictions.keys())

# Инициализация пустого списка для хранения предсказаний
coco_results = []

loading annotations into memory...
Done (t=1.34s)
creating index...
index created!


In [ ]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [ ]:
# Load COCO annotations
coco = COCO('/content/annotations/annotations/instances_val2017.json')
annotations = coco.loadAnns(coco.getAnnIds())


loading annotations into memory...
Done (t=1.09s)
creating index...
index created!


In [ ]:
def calculate_metrics_fast(pred_bboxes, pred_labels, pred_scores, image_id, coco):

    # Получение аннотаций для изображения
    ann_ids = coco.getAnnIds(imgIds=image_id)
    anns = coco.loadAnns(ann_ids)

    #classes_of_interest = [1]

    # Filter annotations for the classes of interest
    #anns = [ann for ann in anns if ann['category_id'] in classes_of_interest]

    # Создание списка предсказанных аннотаций в формате COCO
    pred_anns = []
    for bbox, label, score in zip(pred_bboxes, pred_labels, pred_scores):
        x1, y1, x2, y2 = bbox
        w = x2 - x1
        h = y2 - y1
        pred_ann = {
            'image_id': image_id,
            'category_id': label,
            'bbox': [x1, y1, w, h],
            'score': score
        }
        pred_anns.append(pred_ann)
    print('pred_anns: ', pred_anns)
    # Создание объекта COCOeval для оценки предсказанных аннотаций
    coco_dt = coco.loadRes(pred_anns)
    coco_eval = COCOeval(coco, coco_dt, 'bbox')
    coco_eval.params.imgIds = [image_id]
    #coco_eval.params.catIds = classes_of_interest
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    # Извлечение метрик точности
    metrics = {
        'AP': coco_eval.stats[0],
        'AP50': coco_eval.stats[1],
        'AP75': coco_eval.stats[2],
        'APs': coco_eval.stats[3],
        'APm': coco_eval.stats[4],
        'APl': coco_eval.stats[5],
        'AR1': coco_eval.stats[6],
        'AR10': coco_eval.stats[7],
        'AR100': coco_eval.stats[8],
        'ARs': coco_eval.stats[9],
        'ARm': coco_eval.stats[10],
        'ARl': coco_eval.stats[11]
    }

    return metrics

In [ ]:
annotation_path = '/content/annotations/annotations/instances_val2017.json'

In [ ]:
from tqdm import tqdm

In [ ]:
metrics_map = []
# Инициализация COCO API
coco = COCO(annotation_path)
#for image_id in tqdm(image_ids):
for image_id in image_ids:
    boxes = ensemble_predictions[image_id][0]
    integer_list_labels = ensemble_predictions[image_id][1]
    scores = ensemble_predictions[image_id][2]

    # Вычисление метрик точности
    #def calculate_metrics(pred_bboxes, pred_labels, pred_scores, image_id, annotation):
    metrics = calculate_metrics_fast(boxes, integer_list_labels, scores, image_id, coco)
    metrics_map.append(metrics['APm'])

    # Вывод метрик точности
    print("Metrics:", metrics)


loading annotations into memory...
Done (t=0.71s)
creating index...
index created!
pred_anns:  [{'image_id': 139, 'category_id': 62, 'bbox': [291.41454696655273, 216.71519422531128, 61.4335823059082, 100.85725486278534], 'score': 0.9965628981590271}, {'image_id': 139, 'category_id': 62, 'bbox': [371.1823272705078, 216.47143507003784, 54.47673797607422, 100.25326538085938], 'score': 0.9920255541801453}, {'image_id': 139, 'category_id': 72, 'bbox': [6.433684825897217, 168.40824902057648, 148.2906937599182, 96.3867119550705], 'score': 0.9919740557670593}, {'image_id': 139, 'category_id': 64, 'bbox': [230.8977508544922, 174.7799990773201, 35.86933135986328, 39.05604404211044], 'score': 0.9845151901245117}, {'image_id': 139, 'category_id': 1, 'bbox': [425.2524185180664, 159.8454596400261, 39.59358215332031, 139.64511305093765], 'score': 0.9838053584098816}, {'image_id': 139, 'category_id': 85, 'bbox': [448.84082794189453, 121.13229078054428, 11.988754272460938, 21.34777933359146], 'score': 

In [ ]:
print(sum(metrics_map) / len(metrics_map))

-0.1216451191547726


In [ ]:
def get_image_and_annotation_by_id(image_id, annotation):
    # Инициализация COCO API
    coco = COCO(annotation)

    # Получение информации об изображении по ID
    img_info = coco.loadImgs(image_id)[0]
    image_path = 'images/val2017/' + img_info['file_name']

    # Загрузка изображения
    image = io.imread(image_path)

    # Получение аннотаций для изображения
    ann_ids = coco.getAnnIds(imgIds=image_id)
    anns = coco.loadAnns(ann_ids)

    return image, anns

In [ ]:
def draw_predictions_on_image(image, pred_bboxes, pred_labels, pred_scores, threshold=0.2):
    # Отрисовка ограничивающих рамок на изображении
    image_copy = image.copy()
    for bbox, label, score in zip(pred_bboxes, pred_labels, pred_scores):
        print(score)
        if score >= threshold:
            x1, y1, x2, y2 = map(int, bbox)
            print('x1, y1, x2, y2: ', x1, y1, x2, y2)
            print('label: ', label)
            #print('cat: ', cats[int(label - 1)]['name'])
            cv2.rectangle(image_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)
            #cv2.putText(image_copy, cats[int(label-1)]['name'] + '-' + str(label), (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(image_copy, str(label), (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image_copy

In [ ]:
image, annotation_coco = get_image_and_annotation_by_id(9483, annotation_path)

loading annotations into memory...
Done (t=0.52s)
creating index...
index created!


In [ ]:
annotation_coco

[{'segmentation': [[551.83,
    349.49,
    578.99,
    238.27,
    493.2,
    214.56,
    478.98,
    210.25,
    458.8,
    304.54]],
  'area': 10765.213200000002,
  'iscrowd': 0,
  'image_id': 9483,
  'bbox': [458.8, 210.25, 120.19, 139.24],
  'category_id': 72,
  'id': 31409},
 {'segmentation': [[87.74,
    122.17,
    69.85,
    137.7,
    42.09,
    148.99,
    36.44,
    168.76,
    46.8,
    239.34,
    51.97,
    251.58,
    61.85,
    256.75,
    61.85,
    326.72,
    65.15,
    340.84,
    55.74,
    480.0,
    89.62,
    479.98,
    106.56,
    374.1,
    109.85,
    379.28,
    110.79,
    477.16,
    156.91,
    480.0,
    143.26,
    469.16,
    140.91,
    455.51,
    141.38,
    433.86,
    147.03,
    387.33,
    154.09,
    338.02,
    167.26,
    312.13,
    162.09,
    266.96,
    175.26,
    238.3,
    175.26,
    227.01,
    177.14,
    216.66,
    165.85,
    187.09,
    158.32,
    152.26,
    142.32,
    141.98,
    123.97,
    135.4,
    123.5,
    127.4,
  

In [ ]:
import random

In [ ]:
image_height, image_width = 426, 640

# Extract bounding box coordinates, class labels, and scores
bboxes = []
labels = []
scores = []

for annotation in annotation_coco:
    x, y, width, height = annotation['bbox']
    bbox = [(x / image_width)*random.uniform(0.9, 1.1), (y / image_height)*random.uniform(0.9, 1.1), ((x + width) / image_width)*random.uniform(0.9, 1.1), ((y + height) / image_height)*random.uniform(0.9, 1.1)]
    bboxes.append(bbox)
    labels.append(annotation['category_id'])
    scores.append(random.uniform(0.7, 1))

# Print the converted annotation
print("Bounding Boxes:", bboxes)
print("Class Labels:", labels)
print("Scores:", scores)

Bounding Boxes: [[0.7218856207664132, 0.5231772032559651, 0.9508303267152424, 0.7395407582636089], [0.05594027575209412, 0.14784621275582346, 0.2699934117297278, 1.1937818121673105], [0.597103838385913, 0.8068986761380244, 0.6460162767873612, 0.9278572395270777], [0.4588003369818796, 0.7600547355488305, 0.6877630422253965, 0.8754857693099112], [0.41598403455418653, 0.7099577108634111, 0.41789981810684584, 0.7522510207344185], [0.3397854519765497, 0.6718576236455731, 0.36139600930772947, 0.6514484894060837]]
Class Labels: [72, 1, 74, 76, 72, 72]
Scores: [0.9889574374932701, 0.9710036595855772, 0.950489577554712, 0.7068264493781526, 0.9440401779937222, 0.9108045136261348]


In [ ]:
boxes = faster_rcnn_predictions[9483]['bboxes']
np_boxes = np.array(boxes)
# Конвертируем координаты ограничивающих рамок обратно в пиксели
image_height, image_width = 426, 640
np_boxes[:, 0] *= image_width
np_boxes[:, 1] *= image_height
np_boxes[:, 2] *= image_width
np_boxes[:, 3] *= image_height

integer_list_labels = faster_rcnn_predictions[9483]['labels']
scores = faster_rcnn_predictions[9483]['scores']

image_with_predictions = draw_predictions_on_image(image, np_boxes, integer_list_labels, scores)

0.9997381567955017
x1, y1, x2, y2:  40 61 179 417
label:  1
0.9961842894554138
x1, y1, x2, y2:  306 281 423 323
label:  76
0.9925118684768677
x1, y1, x2, y2:  375 321 409 335
label:  74
0.9868755340576172
x1, y1, x2, y2:  450 187 577 306
label:  72
0.9844557046890259
x1, y1, x2, y2:  108 126 128 188
label:  32
0.9833360910415649
x1, y1, x2, y2:  301 67 377 211
label:  1
0.7763792276382446
x1, y1, x2, y2:  391 166 460 239
label:  62
0.7324556112289429
x1, y1, x2, y2:  301 171 344 233
label:  62
0.6955138444900513
x1, y1, x2, y2:  265 175 362 240
label:  62
0.6929690837860107
x1, y1, x2, y2:  339 175 431 247
label:  62
0.6851582527160645
x1, y1, x2, y2:  164 173 188 254
label:  62
0.6745026707649231
x1, y1, x2, y2:  164 231 306 299
label:  73
0.6052536368370056
x1, y1, x2, y2:  282 172 417 248
label:  62
0.5826508402824402
x1, y1, x2, y2:  260 182 309 229
label:  62
0.5797935128211975
x1, y1, x2, y2:  164 226 303 294
label:  72


In [ ]:
cv2.imwrite('test_faster_rcnn_predictions_9483.jpg', image_with_predictions)

True

In [ ]:
boxes = retinanet_predictions[9483]['bboxes']
np_boxes = np.array(boxes)
# Конвертируем координаты ограничивающих рамок обратно в пиксели
image_height, image_width = 426, 640
np_boxes[:, 0] *= image_width
np_boxes[:, 1] *= image_height
np_boxes[:, 2] *= image_width
np_boxes[:, 3] *= image_height

integer_list_labels = faster_rcnn_predictions[9483]['labels']
scores = faster_rcnn_predictions[9483]['scores']

image_with_predictions = draw_predictions_on_image(image, np_boxes, integer_list_labels, scores)

0.9997381567955017
x1, y1, x2, y2:  38 55 178 422
label:  1
0.9961842894554138
x1, y1, x2, y2:  375 321 409 336
label:  76
0.9925118684768677
x1, y1, x2, y2:  306 281 426 325
label:  74
0.9868755340576172
x1, y1, x2, y2:  456 185 578 310
label:  72
0.9844557046890259
x1, y1, x2, y2:  305 65 374 208
label:  32


In [ ]:
cv2.imwrite('test_retina_predictions_9483.jpg', image_with_predictions)

True

In [ ]:
def draw_annotation_on_image_by_id(image_id, annotation, output_path):
    # Инициализация COCO API
    coco = COCO(annotation)

    # Получение информации об изображении по ID
    img_info = coco.loadImgs(image_id)[0]
    image_path = 'images/val2017/' + img_info['file_name']

    # Загрузка изображения
    image = io.imread(image_path)

    # Получение аннотаций для изображения
    ann_ids = coco.getAnnIds(imgIds=image_id)
    anns = coco.loadAnns(ann_ids)

    # Отрисовка аннотаций на изображении
    for ann in anns:
        bbox = ann['bbox']
        category_id = ann['category_id']
        category_info = coco.loadCats(category_id)[0]
        category_name = category_info['name']

        x, y, w, h = map(int, bbox)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, category_name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Сохранение изображения с аннотацией
    io.imsave(output_path, image)

In [ ]:
draw_annotation_on_image_by_id(9483, annotation_path, output_path = '/content/0000000009483_annotated.jpg')

loading annotations into memory...
Done (t=0.86s)
creating index...
index created!
